In [0]:
%matplotlib inline

In [0]:
!pip install --upgrade efficientnet-pytorch

In [0]:
!rm -r Tiny-ImageNet-C
!rm Tiny-ImageNet-C.tar
!curl https://zenodo.org/record/2536630/files/Tiny-ImageNet-C.tar -o tiny-imagenet-c.tar
!tar -xf tiny-imagenet-c.tar

In [0]:
from __future__ import print_function 
from __future__ import division
from efficientnet_pytorch import EfficientNet
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import io
import pandas as pd
import glob
import os
from shutil import move
from os.path import join
from os import listdir, rmdir
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True, advprop=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes) 
        input_size = 224

    elif model_name == "inception":
        """ Inception v3 
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    elif model_name.startswith("efficientnet"):
        if use_pretrained:
            model_ft = EfficientNet.from_pretrained(model_name, num_classes=num_classes, advprop=advprop)
        else:
            model_ft = EfficientNet.from_name(model_name)
            num_ftrs = model_ft._fc.in_features
            model_ft._fc = nn.Linear(num_ftrs, num_classes)
        set_parameter_requires_grad(model_ft, feature_extract)
        input_sizes = {
            'efficientnet-b0': 224,
            'efficientnet-b1': 240,
            'efficientnet-b2': 260,
            'efficientnet-b3': 300,
            'efficientnet-b4': 380,
            'efficientnet-b5': 456,
            'efficientnet-b6': 528,
            'efficientnet-b7': 600,
            }
        input_size = input_sizes[model_name]

    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size

def evaluate_model(model, dataloader, criterion):
    model.eval()   # Set model to evaluate mode

    running_loss = 0.0
    running_corrects = 0
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            # Get model outputs and calculate loss
            # Special case for inception because in training it has an auxiliary output. In train
            #   mode we calculate the loss by summing the final output and the auxiliary output
            #   but in testing we only consider the final output.
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
    
    loss = running_loss / len(dataloader.dataset)
    acc = running_corrects.double() / len(dataloader.dataset)

    print('Validation Loss: {:.4f} Acc: {:.4f}'.format(loss, acc))

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [0]:
data_dir = "./tiny-imagenet-200"
model_name = "efficientnet-b4"
model_output = "{}_chkpt_last_epoch".format(model_name)
num_classes = 200
batch_size = 16 # change if CUDA memory low
feature_extract = False

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=15),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
print("Initializing Datasets and Dataloaders...")

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Send the model to GPU
model_ft = model_ft.to(device)

model_ft.load_state_dict(torch.load(F"/content/drive/My Drive/{model_output}"))

In [0]:
for path in glob.glob("./Tiny-ImageNet-C/*/[1-5]/"):
  print('EVALUATING MODEL ON {}'.format(path))
  imagenet_c = datasets.ImageFolder(path, data_transforms['val'])
  # Create training and validation dataloaders
  dataloader_imagenet_c = torch.utils.data.DataLoader(imagenet_c, batch_size=batch_size, shuffle=True, num_workers=4)
  evaluate_model(model_ft, dataloader_imagenet_c, criterion)
  print()